# Middleware: Human In The Loop
人机回路（HITL）中间件允许您为Agent工具调用添加人工监督。当模型提议一个可能需要审核的操作时 —— 例如，写入文件或执行 SQL—— 中间件可以暂停执行并等待决策。

它通过将每个工具调用与可配置的策略进行比对来实现这一点。如果需要干预，中间件会发出一个中断信号来停止执行。图状态会使用 LangGraph 的持久化层进行保存，因此执行可以安全地暂停并在稍后继续。

然后由人工决策决定下一步的操作：操作可以原样批准（approve）、在运行前进行修改（edit），或附带反馈被拒绝（reject）。

<img src="./assets/LC_HITL.png" width="300">

# Interrupt 决策类型
中间件定义了三种人类可以响应中断的方式：

|决策类型	|描述	|示例用例|
|----|----|----|
|✅approve|操作按原样批准并执行，无需更改|按原文发送邮件草稿|
|✏️edit|工具调用已修改后执行|发送邮件前更改收件人|
|❌reject|工具调用被拒绝，并在对话中添加了说明|拒绝邮件草稿并解释如何重写它|


每种工具可用的决策类型取决于你在 `interrupt_on` 中配置的策略。当多个工具调用同时被暂停时，每个操作都需要单独的决策。决策必须按照中断请求中操作出现的顺序提供。



## 初始化
加载并检查所需的环境变量

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env, doublecheck_pkgs

# 从 .env 加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")  # 检查环境变量

DASHSCOPE_API_KEY=****931f
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****ef8f
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****s-56
LANGSMITH_ENDPOINT=****.com


In [4]:
#导入实例数据库
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

#定义运行时上下文，为Agent和tools提供数据库访问。
from dataclasses import dataclass

# 定义上下文结构以支持依赖注入
@dataclass
class RuntimeContext:
    """运行时上下文，保存数据库连接供Agent和tools使用。"""
    db: SQLDatabase

from langchain.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """执行SQL查询并返回结果。"""
    runtime = get_runtime(RuntimeContext)   # 取出运行时上下文
    db = runtime.context.db                 # 获取数据库连接
    try:
        return db.run(query)                # 进行数据库查询
    except Exception as e:
        return f"执行查询时出错: {e}"
    

SYSTEM_PROMPT = """你是一名严谨的 SQLite 分析师。

规则：
- 逐步思考。
- 如需数据,请使用名为“execute_sql”的工具并仅执行一个 SELECT 查询。
- 仅读取数据，禁止执行 INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE 操作。
- 输出结果最多限制为 5 行，除非用户明确另有要求。
- 如果工具返回“执行查询时出错:”,请修改 SQL 语句并再次尝试。
- 优先明确列出列名；避免使用 SELECT * 。
"""


## Middleware

In [5]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_qwq import ChatQwen
import os
model=ChatQwen(
    model="qwen3-max", 
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    )
from langchain.agents import create_agent

agent=create_agent(
    model=model,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    context_schema=RuntimeContext,
    middleware=[
        HumanInTheLoopMiddleware( #工具的名称。当agent尝试调用这个工具时，中间件会拦截
            interrupt_on={"execute_sql": {"allowed_decisions": ["approve", "reject"]}},
        ),#这个工具允许的人工决定类型
    ],
)

In [6]:
from langgraph.types import Command

question = "What are the names of all the employees?"
#所有员工的姓名是什么？
config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

if "__interrupt__" in result:# 检查执行结果中是否存在中断
    # 获取最后一个中断对象的值
    # result['__interrupt__'] 是一个列表，包含所有中断
     # ['action_requests'] 获取需要审核的操作列表
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )
    print(result['__interrupt__'][-1])
    # 调用agent继续执行，传入人工的审核决定
    result = agent.invoke(
        Command(# 使用Command对象恢复暂停的对话
            resume={# resume字段包含人工的决定列表
                "decisions": [
                    {
                        # 决定类型：拒绝该操作
                        "type": "reject",
                        # 拒绝的原因（会作为反馈发给agent）
                        "message": "the database is offline."#数据库已离线。
                    }
                ]
            }
        ),
        config=config,  # 使用相同的线程 ID 以恢复暂停的对话
        context=RuntimeContext(db=db),
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT name FROM employees;'}
Interrupt(value={'action_requests': [{'name': 'execute_sql', 'args': {'query': 'SELECT name FROM employees;'}, 'description': "Tool execution requires approval\n\nTool: execute_sql\nArgs: {'query': 'SELECT name FROM employees;'}"}], 'review_configs': [{'action_name': 'execute_sql', 'allowed_decisions': ['approve', 'reject']}]}, id='b28866be11cf2e8aa17c4999084770a7')
--------------------------------------------------------------------------------
It seems the database is currently offline, so I'm unable to retrieve the names of the employees. Please check the database connection and try again later.


In [7]:
config = {"configurable": {"thread_id": "3"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

while "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )
    
    result = agent.invoke(
        Command(
            resume={"decisions": [{"type": "approve"}]}
        ),
        config=config,  # 使用相同的线程 ID 以恢复暂停的对话
        context=RuntimeContext(db=db),
    )

for msg in result["messages"]:
    msg.pretty_print()

--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT name FROM employees;'}
--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT name FROM sqlite_master WHERE type='table';"}
--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT FirstName, LastName FROM Employee;'}
================================ Human Message =================================

What are the names of all the employees?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_b904675459f943a48299e7b1)
 Call ID: call_b904675459f943a48299e7b1
  Args:
    query: SELECT name FROM employees;
================================= Tool Me